# Process Deforestation Data from Hansen et al.

In this notebook we will process the tree cover and deforestation data from Hansen et al. This map data is divided into 504 granules of 10x10 degrees, spanning from 60 degrees South to 80 degrees North

In [1]:
import numpy as np
import rasterio
import PIL
from PIL import Image
from pathlib import Path
import pickle
import json
import multiprocessing

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
Image.MAX_IMAGE_PIXELS = 2000000000  # Increase PIL image load limit

In [3]:
longitude_range = (-180, 180)
latitude_range = (-60, 80)

pixels_per_degree = 3

longitude_size = (longitude_range[1] - longitude_range[0]) * pixels_per_degree
latitude_size = (latitude_range[1] - latitude_range[0]) * pixels_per_degree
granule_size = longitude_size / 36

km_to_px_EC = 2 * np.pi * 6371 / 360 / pixels_per_degree

img_common_size = (latitude_size, longitude_size)
print(img_common_size)
print(f"Each pixel will have a side of {km_to_px_EC:.3f} km at the ecuator.")
print(f"Each granule will have {granule_size:.0f} pixels to a side.")

(420, 1080)
Each pixel will have a side of 37.065 km at the ecuator.
Each granule will have 30 pixels to a side.


In [17]:
def granule_from_tif(tif_location, granule_size=30):
    locator = (tif_location).stem[-8:]

    with rasterio.open(tif_location) as dataset_reader:
        # Upscale factor for achieving appropriate longitude dimension
        upscale_factor = 1/(dataset_reader.width / granule_size)
        # Downscale the dataset. Avg scaling to retain integer info
        array = dataset_reader.read(
            out_shape=(
                dataset_reader.count,
                int(dataset_reader.height * upscale_factor),
                int(dataset_reader.width * upscale_factor)
            ),
            resampling=rasterio.enums.Resampling.average
        ).squeeze()
    
    return (array, locator)

In [18]:
def integrate_granule_to_base(base, granule, locator, lat_range=(-60, 80), granule_size=30):
    lat = int(locator[:2]) * (1 if locator[2] == "N" else -1)
    lon = int(locator[4:7]) * (1 if locator[7] == "E" else -1)
    
    lat_degs = lat_range[1] - lat_range[0]
    
    lat_min_id = int((80 - lat) / lat_degs * base.shape[0])
    lat_max_id = lat_min_id + granule_size
    lon_min_id = int((lon + 180) / 360 * base.shape[1])
    lon_max_id = lon_min_id + granule_size
        
    base[lat_min_id:lat_max_id, lon_min_id:lon_max_id] = granule
    
    return base

In [34]:
data_folder = Path("../data/raw/hansen")
output_folder = Path("../data/processed/hansen")
filename="Hansen_GFC-2018-v1.6_treecover2000_50N_010W.tif"

In [21]:
base = np.zeros(img_common_size)

In [23]:
list(data_folder.glob("*tree*.tif"))

[PosixPath('../data/raw/hansen/Hansen_GFC-2018-v1.6_treecover2000_50N_020E.tif'),
 PosixPath('../data/raw/hansen/Hansen_GFC-2018-v1.6_treecover2000_50N_000E.tif'),
 PosixPath('../data/raw/hansen/Hansen_GFC-2018-v1.6_treecover2000_50N_010E.tif'),
 PosixPath('../data/raw/hansen/Hansen_GFC-2018-v1.6_treecover2000_50N_010W.tif')]

In [40]:
def save_outputs(array, name, location, value_type="real", round_to=3):
    with open(location/f"{name}.pickle", "wb") as file:
        pickle.dump(array, file)
    array_to_json(array, location/f"{name}.json", value_type=value_type, round_to=round_to)
    plt.imsave(location/f"{name}.jpg", array, cmap="inferno")

In [41]:
def process_all(layer, input_folder, output_folder, img_size=(420, 1080)):
    tif_files = list(input_folder.glob(f"*{layer}*.tif"))
    
    max_processes = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(max_processes)
    results = pool.map(granule_from_tif, tif_files)
    
    base = np.zeros(img_size)
    for granule, locator in results:
        base = integrate_granule_to_base(base, granule, locator)
    array = base
    save_outputs(array, layer, output_folder, value_type="real", round_to=3)
    return array

In [44]:
def array_to_json(array, destination, value_name="value", value_type="real", 
                  round_to=3, lon_range=(-180, 180), lat_range=(-60, 80), threshold=0, normalize_by=1.):
    data = {}
    fields = [
        {"name": "latitude", "format": "", "type": "real"},
        {"name": "longitude", "format": "", "type": "real"},
        {"name": value_name, "format": "", "type": value_type},
    ]
    data["fields"] = fields
    data["rows"] = array_to_coords(array, 
        round_to=round_to, 
        lon_range=lon_range, 
        lat_range=lat_range, 
        threshold=threshold,
        normalize_by=normalize_by,
    )
    with open(destination, 'w') as file:
        json.dump(data, file)

In [46]:
def array_to_coords(array, round_to=3, lon_range=(-180, 180), lat_range=(-60, 80), threshold=0, normalize_by=1.):
    norm = array/normalize_by
    lats = np.arange(lat_range[1], lat_range[0], -(lat_range[1] - lat_range[0])/array.shape[0])
    lons = np.arange(lon_range[0], lon_range[1],  (lon_range[1] - lon_range[0])/array.shape[1])
    rows = list()
    rows = [[round(float(lats[i]), round_to), 
             round(float(lons[j]), round_to), 
             round(float(norm[i, j]), round_to)]
            for j in range(array.shape[1]) 
            for i in range(array.shape[0])
            if array[i, j] > threshold]  # Do not store values below or equal to the threshold (e.g. zeroes)
    return rows

In [47]:
%%time
array = process_all("treecover2000", data_folder, Path("../data/processed/forest"))

CPU times: user 791 ms, sys: 155 ms, total: 946 ms
Wall time: 7.83 s
